In [1]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


/Users/ted/dev/stylegan3/.venv/lib/python3.9/site-packages/torch/_tensor_str.py:137: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:335.)
  nonzero_finite_vals = torch.masked_select(


In [1]:
# https://www.linkedin.com/pulse/how-use-gpu-tensorflow-pytorch-libraries-macbook-pro-m2apple-kashyap/

# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# StyleGAN3 from NVIDIA

**Notes**
* To see the original code from NVIDIA [Check here](https://github.com/NVlabs/stylegan3)
* We are using a pretrained model and fine-tuning on top of it.
* If you come across bugs please post them in [Discord](https://discord.com/invite/awREd7EtMA)

---

If you find this notebook useful, consider signing up for my [Code Sprout Newsletter](https://codesprout.substack.com/welcome) or [Check my links](https://shyambv.bio.link/)

Medium article related to it is mentioned []()

# Setup

In [2]:
# !pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install torch==1.13.0
# torchvision torchaudio -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 50.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.0 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.0 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torc

In [3]:
!pip install einops ninja gdown torch_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 897.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_utils: filename=torch_utils-0.1.2-py3-none-any.whl size=6186 sha256=07ab98df650e042786b02bf17c4436ef78dc83a88cf0c5e313a417b7287957aa
  Stored in directory: /root/.cache/pip/wheels/76/08/f0/378d1fe4aac5aa1e21483918d70b7e3428c4faaac0abda4e15
Successfully built torch_utils


In [4]:
import os
if os.path.isdir('/content/drive/MyDrive/WIP/stylegan3/'):
    %cd '/content/drive/MyDrive/WIP/stylegan3/'
else:
    # !git clone https://github.com/bvshyam/stylegan3.git /content/drive/MyDrive/WIP/stylegan3/
    %cd '/content/drive/MyDrive/WIP/stylegan3/'


/content/drive/MyDrive/WIP/stylegan3


# Generate images

In [5]:
#model_path = '/content/drive/MyDrive/WIP/stylegan3/results/00002-stylegan3-t-artimages-512x512-gpus1-batch16-gamma50/network-snapshot-000001.pkl'

In [6]:
#Random generation of images using model created

#!python /content/drive/MyDrive/WIP/stylegan3/gen_images.py --outdir=out --trunc=1 --seeds=40 \
    # --network=$model_path

Check-out other articles [Medium](https://bvshyam.medium.com/)

In [3]:
import torch
import pickle
import os
import PIL.Image
from IPython.display import Image
import matplotlib.pyplot as plt
import IPython.display

"""Generate images using pretrained network pickle."""

import os
import re
from typing import List, Optional, Tuple, Union

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy

from pathlib import Path



/Users/ted/dev/stylegan3/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:

# model_path = '/content/drive/MyDrive/dev/arco/training/00021-stylegan3-r-rose01-blobs-square-gpus1-batch8-gamma6.6/network-snapshot-000160.pkl'
model_path = '/Users/ted/dev/stylegan3/00022-stylegan3-r-240523_01-rose01-YFlips-6min-gpus1-batch8-gamma6.6/network-snapshot-000072.pkl'
outdir = '/Volumes/2024-May-Ted-Moore/videos/rose/stylegan/240525_02-roseYFlips-seeds-1000-1499-pkl=72'

torch.device('mps')
#----------------------------------------------------------------------------

assert os.path.isfile(model_path)

with open(model_path, 'rb') as f:
    G = pickle.load(f)['G_ema'].to(mps_device) #cuda()  # torch.nn.Module

def parse_range(s: Union[str, List]) -> List[int]:
    '''Parse a comma separated list of numbers or ranges and return a list of ints.
    Example: '1,2,5-10' returns [1, 2, 5, 6, 7]
    '''
    if isinstance(s, list): return s
    ranges = []
    range_re = re.compile(r'^(\d+)-(\d+)$')
    for p in s.split(','):
        m = range_re.match(p)
        if m:
            ranges.extend(range(int(m.group(1)), int(m.group(2))+1))
        else:
            ranges.append(int(p))
    return ranges

#----------------------------------------------------------------------------

def parse_vec2(s: Union[str, Tuple[float, float]]) -> Tuple[float, float]:
    '''Parse a floating point 2-vector of syntax 'a,b'.
    Example:
        '0,1' returns (0,1)
    '''
    if isinstance(s, tuple): return s
    parts = s.split(',')
    if len(parts) == 2:
        return (float(parts[0]), float(parts[1]))
    raise ValueError(f'cannot parse 2-vector {s}')

#----------------------------------------------------------------------------

def make_transform(translate: Tuple[float,float], angle: float):
    m = np.eye(3)
    s = np.sin(angle/360.0*np.pi*2)
    c = np.cos(angle/360.0*np.pi*2)
    m[0][0] = c
    m[0][1] = s
    m[0][2] = translate[0]
    m[1][0] = -s
    m[1][1] = c
    m[1][2] = translate[1]
    return m

#----------------------------------------------------------------------------

# @click.command()
# @click.option('--network', 'network_pkl', help='Network pickle filename', required=True)
# @click.option('--seeds', type=parse_range, help='List of random seeds (e.g., \'0,1,4-6\')', required=True)
# @click.option('--trunc', 'truncation_psi', type=float, help='Truncation psi', default=1, show_default=True)
# @click.option('--class', 'class_idx', type=int, help='Class label (unconditional if not specified)')
# @click.option('--noise-mode', help='Noise mode', type=click.Choice(['const', 'random', 'none']), default='const', show_default=True)
# @click.option('--translate', help='Translate XY-coordinate (e.g. \'0.3,1\')', type=parse_vec2, default='0,0', show_default=True, metavar='VEC2')
# @click.option('--rotate', help='Rotation angle in degrees', type=float, default=0, show_default=True, metavar='ANGLE')
# @click.option('--outdir', help='Where to save the output images', type=str, required=True, metavar='DIR')
# def generate_images(
#     network_pkl: str,
#     seeds: List[int],
#     truncation_psi: float,
#     noise_mode: str,
#     outdir: str,
#     translate: Tuple[float,float],
#     rotate: float,
#     class_idx: Optional[int]
# ):
"""Generate images using pretrained network pickle.
Examples:
\b
# Generate an image using pre-trained AFHQv2 model ("Ours" in Figure 1, left).
python gen_images.py --outdir=out --trunc=1 --seeds=2 \\
    --network=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl
\b
# Generate uncurated images with truncation using the MetFaces-U dataset
python gen_images.py --outdir=out --trunc=0.7 --seeds=600-605 \\
    --network=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-metfacesu-1024x1024.pkl
"""

# =============== setting params manually instead of with click ==================
class_idx = None
translate = (0,0)
noise_mode = 'const'
rotate = 0
truncation_psi = 1
# ================================================================================

Path(outdir).mkdir(exist_ok=True)

print('Loading networks from "%s"...' % model_path)
device = torch.device('mps')
with dnnlib.util.open_url(model_path) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

# os.makedirs(outdir, exist_ok=True)

# Labels.
label = torch.zeros([1, G.c_dim], device=device)
if G.c_dim != 0:
    if class_idx is None:
        raise click.ClickException('Must specify class label with --class when using a conditional network')
    label[:, class_idx] = 1
else:
    if class_idx is not None:
        print ('warn: --class=lbl ignored when running on an unconditional network')


# Generate images.
seeds_start = 1000
n_seeds = 500

for i, seed in enumerate(range(seeds_start,seeds_start + n_seeds)):
    # print('Generating image for seed %d (%d/%d) ...' % (seed, seed_idx, len(seeds)))
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(torch.float32).to(device)
    # print(z.shape)
    # print(z)

    # Construct an inverse rotation/translation matrix and pass to the generator.  The
    # generator expects this matrix as an inverse to avoid potentially failing numerical
    # operations in the network.
    if hasattr(G.synthesis, 'input'):
        m = make_transform(translate, rotate)
        m = np.linalg.inv(m)
        G.synthesis.input.transform.copy_(torch.from_numpy(m))

    img = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    save_path = os.path.join(outdir,f'seed{seed:04d}.png')
    PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(save_path)

    print(f'{i} / {n_seeds}\t{save_path}')

    # display:
    # plt.axis('off')
    # img = img.reshape((img.shape[1],img.shape[2],img.shape[3]))
    # plt.imshow(img.cpu())
    # plt.show()

Loading networks from "/Users/ted/dev/stylegan3/00022-stylegan3-r-240523_01-rose01-YFlips-6min-gpus1-batch8-gamma6.6/network-snapshot-000072.pkl"...
0 / 500	/Volumes/2024-May-Ted-Moore/videos/rose/stylegan/240525_02-roseYFlips-seeds-1000-1499-pkl=72/seed1000.png
1 / 500	/Volumes/2024-May-Ted-Moore/videos/rose/stylegan/240525_02-roseYFlips-seeds-1000-1499-pkl=72/seed1001.png
2 / 500	/Volumes/2024-May-Ted-Moore/videos/rose/stylegan/240525_02-roseYFlips-seeds-1000-1499-pkl=72/seed1002.png
3 / 500	/Volumes/2024-May-Ted-Moore/videos/rose/stylegan/240525_02-roseYFlips-seeds-1000-1499-pkl=72/seed1003.png
4 / 500	/Volumes/2024-May-Ted-Moore/videos/rose/stylegan/240525_02-roseYFlips-seeds-1000-1499-pkl=72/seed1004.png


KeyboardInterrupt: 